# Testing FeatureCounts Behavior

Background: SHAREseq RNA libraries are fr-stranded (read 1 is the original mRNA sequence). To ensure that the featureCounts tool is assigning features to aligned reads properly, we are testing a few scenarios as described below by creating synthetic reads.

Scenarios to test:

1. pos
    - description: read that aligns to the correct strand of an annotation
2. neg
    - description: read that aligns to the opposite strand of an annotation
3. intron
    - description: read that aligns to just an intron
4. splice
    - description: read that aligns across an exon splice junction
5. overlap1
    - description: read that aligns overlapping two transcripts for the same gene
6. overlap2
    - description: read that aligns overlapping with two transcripts from different genes (opposite strands)

Our test locus on chr12:

We see several isoforms of ADGRD1 and its overlapping antisense gene ADGRD1-AS1

splice_a and splice_b are the first and second parts of our spliced read

<img src='./test_featureCounts.svg'/>

## Create mini STAR reference
We make a STAR reference containing just chr12. 

This takes about 3GB of disk space for the outputs and 5 minutes to build the index single-threaded (fewer with the 6 threads shown here)

In [1]:
%%bash
# Get a subsetted gtf file with just the lines we care about
# sed -n '2094578,2094868p' /oak/stanford/groups/wjg/bliu/resources/gtf/gencode.v41.annotation.BPfiltered.gtf > genes.gtf

# Clean up any previous outputs
rm -f chr12.fa*
rm -rf star_chr12
rm -rf /tmp/star_chr12

# Download chr12 reference genome
wget --quiet https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr12.fa.gz
gunzip chr12.fa.gz

# Generate STAR index
mkdir star_chr12
STAR \
        --runMode genomeGenerate \
        --runThreadN 6 \
        --genomeDir star_chr12 \
        --genomeFastaFiles chr12.fa \
        --outFileNamePrefix star_chr12 \
        --outTmpDir /tmp/star_chr12 \
        --sjdbGTFfile genes.gtf \
        --genomeSAindexNbases 12

	STAR --runMode genomeGenerate --runThreadN 6 --genomeDir star_chr12 --genomeFastaFiles chr12.fa --outFileNamePrefix star_chr12 --outTmpDir /tmp/star_chr12 --sjdbGTFfile genes.gtf --genomeSAindexNbases 12
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Feb 20 13:27:30 ..... started STAR run
Feb 20 13:27:30 ... starting to generate Genome files
Feb 20 13:27:41 ..... processing annotations GTF
Feb 20 13:27:41 ... starting to sort Suffix Array. This may take a long time...
Feb 20 13:27:42 ... sorting Suffix Array chunks and saving them to disk...
Feb 20 13:28:17 ... loading chunks from disk, packing SA...
Feb 20 13:28:28 ... finished generating suffix array
Feb 20 13:28:28 ... generating Suffix Array index
Feb 20 13:28:39 ... completed Suffix Array index
Feb 20 13:28:39 ..... inserting junctions into the genome indices
Feb 20 13:28:42 ... writing Genome to disk ...
Feb 20 13:28:42 ... writing Suffix Array to disk ...
Feb 20 1

## Create synthetic test reads
We use bedtools getfasta to create the test file (plus a little hand-editing afterwards to convert to fastq format)

In [2]:
%%bash
bedtools getfasta -fi chr12.fa -bed test.bed -s -name

index file chr12.fa.fai not found, generating...


>splice::chr12:130982012-130982063(+)
TGTATACGCGGGACAATTCCATGACATGGGAGGCCTCCTTCAGCCCCCCAG
>splice::chr12:130987094-130987143(+)
GCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCT
>pos::chr12:130987094-130987194(+)
GCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCTGAAAGTCTACGTCAACGGGACCCTGAGCACCTCTGATCCGAGTGGAAAAGT
>neg::chr12:130987094-130987194(-)
ACTTTTCCACTCGGATCAGAGGTGCTCAGGGTCCCGTTGACGTAGACTTTCAGGCCCTCCTTGGATTTCCATGTAAATAGGACATGAGTCCAATAGGGGC
>intron::chr12:130983000-130983100(+)
CTCATTCTTAAGGGAGGCTgcatggcccagttattaaaaagcgtgaactctggacccaaacaggctggatcaaatcccgcctctggacgttggtagttgt
>overlap1::chr12:130981933-130982033(+)
ATCCCTTCTGCGTATGGGGGACAGGTCATCTCCAATGGGTTCAAAGTCTGCTCCAGCGGTGGCAGAGGCTCTGTGGAGCTGTATACGCGGGACAATTCCA
>overlap2_pos::chr12:130991013-130991113(+)
GAAAGCATGCTTTATTGTCTTCAACGCTGCCAAGCCTCTTCATGACATCCACAGCAAGCCCCGTGGTGAGCAGACACATCTTCCTTGGTCCCCCTTGCTG
>overlap2_neg::chr12:130991013-130991113(-)
CAGCAAGGGGGACCAAGGAAGATGTGTCTGCTCACCACGGGGCTTGCTGTGGATGTCATGAAGAGGCTTGGCAGCGTTGAA

In [3]:
!cat test.fastq

@test_pos
GCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCTGAAAGTCTACGTCAACGGGACCCTGAGCACCTCTGATCCGAGTGGAAAAGT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@test_neg
ACTTTTCCACTCGGATCAGAGGTGCTCAGGGTCCCGTTGACGTAGACTTTCAGGCCCTCCTTGGATTTCCATGTAAATAGGACATGAGTCCAATAGGGGC
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@test_intron
CTCATTCTTAAGGGAGGCTgcatggcccagttattaaaaagcgtgaactctggacccaaacaggctggatcaaatcccgcctctggacgttggtagttgt
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@test_splice
TGTATACGCGGGACAATTCCATGACATGGGAGGCCTCCTTCAGCCCCCCAGGCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@test_overlap1
ATCCCTTCTGCGTATGGGGGACAGGTCATCTCCAATGGGTTCAAAGTCTGCTCCAGCGGTGGCAGAGGCTCTGTGGAGCTGTATACGCGGGACAATTCCA
+
FFFFFFFFFFFFFFFFFFFF

## Align our synthetic test reads

In [4]:
%%bash 
STAR --chimOutType WithinBAM \
    --runThreadN 1 \
    --genomeDir star_chr12 \
    --readFilesIn test.fastq \
    --outFileNamePrefix test \
    --outFilterMultimapNmax 50 \
    --outFilterScoreMinOverLread 0.3 \
    --outFilterMatchNminOverLread 0.3 \
    --outSAMattributes NH HI AS NM MD \
    --outSAMtype BAM Unsorted \
    --outSAMunmapped Within \
    --outSAMstrandField intronMotif \
    --outReadsUnmapped None \
    --outFilterType BySJout \
    --outFilterMismatchNmax 999 \
    --outFilterMismatchNoverReadLmax 0.04 \
    --alignIntronMin 10 \
    --alignIntronMax 1000000 \
    --alignMatesGapMax 1000000 \
    --alignSJoverhangMin 8 \
    --alignSJDBoverhangMin 1 \
    --sjdbScore 1 \
    --limitOutSJcollapsed 5000000 

	STAR --chimOutType WithinBAM --runThreadN 1 --genomeDir star_chr12 --readFilesIn test.fastq --outFileNamePrefix test --outFilterMultimapNmax 50 --outFilterScoreMinOverLread 0.3 --outFilterMatchNminOverLread 0.3 --outSAMattributes NH HI AS NM MD --outSAMtype BAM Unsorted --outSAMunmapped Within --outSAMstrandField intronMotif --outReadsUnmapped None --outFilterType BySJout --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 10 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --limitOutSJcollapsed 5000000
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Feb 20 13:28:53 ..... started STAR run
Feb 20 13:28:53 ..... loading genome
Feb 20 13:28:59 ..... started mapping
Feb 20 13:28:59 ..... finished mapping
Feb 20 13:28:59 ..... finished successfully


In [5]:
%%bash
samtools view -h testAligned.out.bam > aligned.sam
bedtools bamtobed -i aligned.sam

chr12	130987094	130987194	test_pos	255	+
chr12	130987094	130987194	test_neg	255	-
chr12	130983000	130983100	test_intron	255	+
chr12	130982012	130987143	test_splice	255	+
chr12	130981933	130982033	test_overlap1	255	+
chr12	130991013	130991113	test_overlap2_pos	255	+
chr12	130991013	130991113	test_overlap2_neg	255	-


In [6]:
%%bash
# show synthetic reads in sam file
samtools view aligned.sam

test_pos	0	chr12	130987095	255	100M	*	0	0	GCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCTGAAAGTCTACGTCAACGGGACCCTGAGCACCTCTGATCCGAGTGGAAAAGT	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	AS:i:98	NM:i:0	MD:Z:100
test_neg	16	chr12	130987095	255	100M	*	0	0	GCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGCCTGAAAGTCTACGTCAACGGGACCCTGAGCACCTCTGATCCGAGTGGAAAAGT	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	AS:i:98	NM:i:0	MD:Z:100
test_intron	0	chr12	130983001	255	100M	*	0	0	CTCATTCTTAAGGGAGGCTGCATGGCCCAGTTATTAAAAAGCGTGAACTCTGGACCCAAACAGGCTGGATCAAATCCCGCCTCTGGACGTTGGTAGTTGT	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	AS:i:98	NM:i:0	MD:Z:100
test_splice	0	chr12	130982013	255	51M5031N49M	*	0	0	TGTATACGCGGGACAATTCCATGACATGGGAGGCCTCCTTCAGCCCCCCAGGCCCCTATTGGACTCATGTCCTATTTACATGGAAATCCAAGGAGGGC

## run featureCounts

### align to gene, include intronic reads, strand specific (current SHAREseq pipeline strategy)

In [7]:
%%bash
featureCounts -Q 30 -a genes.gtf -t gene -g gene_name -s 1 -o output.genes -R CORE aligned.sam 2> featureCounts.log
cat aligned.sam.featureCounts

test_pos	Assigned	1	ADGRD1
test_neg	Unassigned_NoFeatures	-1	NA
test_intron	Assigned	1	ADGRD1
test_splice	Assigned	1	ADGRD1
test_overlap1	Assigned	1	ADGRD1
test_overlap2_pos	Assigned	1	ADGRD1
test_overlap2_neg	Assigned	1	ADGRD1-AS1


### align to exon, exclude intronic reads, strand specific

In [8]:
%%bash
featureCounts -Q 30 -a genes.gtf -t exon -g gene_name -s 1 -o output.genes -R CORE aligned.sam 2> featureCounts.log
cat aligned.sam.featureCounts

test_pos	Assigned	1	ADGRD1
test_neg	Unassigned_NoFeatures	-1	NA
test_intron	Unassigned_NoFeatures	-1	NA
test_splice	Assigned	1	ADGRD1
test_overlap1	Assigned	1	ADGRD1
test_overlap2_pos	Assigned	1	ADGRD1
test_overlap2_neg	Assigned	1	ADGRD1-AS1


### align to gene, include intronic reads, non stranded

In [9]:
%%bash
featureCounts -Q 30 -a genes.gtf -t gene -g gene_name -o output.genes -R CORE aligned.sam 2> featureCounts.log
cat aligned.sam.featureCounts

test_pos	Assigned	1	ADGRD1
test_neg	Assigned	1	ADGRD1
test_intron	Assigned	1	ADGRD1
test_splice	Assigned	1	ADGRD1
test_overlap1	Assigned	1	ADGRD1
test_overlap2_pos	Unassigned_Ambiguity	-1	NA
test_overlap2_neg	Unassigned_Ambiguity	-1	NA


### align to exon, exclude intronic reads, non stranded

In [10]:
%%bash
featureCounts -Q 30 -a genes.gtf -t exon -g gene_name -o output.genes -R CORE aligned.sam 2> featureCounts.log
cat aligned.sam.featureCounts

test_pos	Assigned	1	ADGRD1
test_neg	Assigned	1	ADGRD1
test_intron	Unassigned_NoFeatures	-1	NA
test_splice	Assigned	1	ADGRD1
test_overlap1	Assigned	1	ADGRD1
test_overlap2_pos	Unassigned_Ambiguity	-1	NA
test_overlap2_neg	Unassigned_Ambiguity	-1	NA
